In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

In [2]:
df_directorio = pd.read_excel("DIRECTORIO DE OFICINAS.xlsx")
df_base = pd.read_excel("BASE CORREOS.xlsx")

In [3]:
df_directorio.head(3)

,Registro,Nombre,Correo,Puesto,División,ID Oficina,Nombre Oficina
0,P015072,"GANOZA HONORES, NATALIE ELIZABETH",NGANOZA@BBVA.COM,(B417)SUB GERENTE DE OFICINA F,DIVISION MIRAFLORES,564,OFICINA PLAZA BUTTERS
1,P016944,"QUISPE MAMANI, LUZ MERI",LMQUISPE@BBVA.COM,(B217)GERENTE DE OFICINA F,DIVISION SUR,763,OFICINA PEDREGAL
2,P016949,"TORRES PAREDES, GUISSEL PIERANGELY",GUISSELPIERANGELY.TORRES@BBVA.COM,(B252)EJECUTIVO BANCA NEGOCIOS 3,DIVISION SUR,763,OFICINA PEDREGAL


In [4]:
df_base.drop(df_base.columns[-1], axis=1, inplace=True)
df_base.head(3)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, eyttus...",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,5,"GCASTROP@BBVA.COM, buzon-procesal-sin-garantia...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com, paul...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA"


In [5]:
print('df_directorio: ', df_directorio.shape)
print('df_base: ', df_base.shape)

df_directorio:  (2944, 7)
df_base:  (333, 7)


In [6]:
df_directorio.isna().sum()

Registro          179
Nombre            179
Correo            181
Puesto              0
División            0
ID Oficina          0
Nombre Oficina    179
dtype: int64

In [7]:
df_base.isna().sum()

OFICINA           0
CORREOS           0
Territorio        1
Nombre Oficina    1
Cargo             1
Registro          1
Nombre            0
dtype: int64

In [8]:
df_directorio.dropna(inplace=True)
df_base.dropna(inplace=True)

print('df_directorio: ', df_directorio.shape)
print('df_base: ', df_base.shape)

df_directorio:  (2763, 7)
df_base:  (332, 7)


In [9]:
columnas_directorio = ['ID Oficina', 'Puesto', 'Correo', 'Registro', 'Nombre']
df_directorio = df_directorio[columnas_directorio]
df_directorio.rename(columns={'ID Oficina': 'OFICINA'}, inplace=True)
df_directorio.head(3)

,OFICINA,Puesto,Correo,Registro,Nombre
0,564,(B417)SUB GERENTE DE OFICINA F,NGANOZA@BBVA.COM,P015072,"GANOZA HONORES, NATALIE ELIZABETH"
1,763,(B217)GERENTE DE OFICINA F,LMQUISPE@BBVA.COM,P016944,"QUISPE MAMANI, LUZ MERI"
2,763,(B252)EJECUTIVO BANCA NEGOCIOS 3,GUISSELPIERANGELY.TORRES@BBVA.COM,P016949,"TORRES PAREDES, GUISSEL PIERANGELY"


In [10]:
df_directorio = df_directorio[df_directorio['Puesto'].str.contains('GERENTE', case=False)].reset_index(drop=True)
df_directorio.head(3)

,OFICINA,Puesto,Correo,Registro,Nombre
0,564,(B417)SUB GERENTE DE OFICINA F,NGANOZA@BBVA.COM,P015072,"GANOZA HONORES, NATALIE ELIZABETH"
1,763,(B217)GERENTE DE OFICINA F,LMQUISPE@BBVA.COM,P016944,"QUISPE MAMANI, LUZ MERI"
2,346,(B415)SUB GERENTE DE OFICINA E,MROCAM@BBVA.COM,P018086,"ROCA MORENO, MALU ESPERANZA"


In [11]:
df_directorio['Puesto'] = df_directorio['Puesto'].replace({
    '(?i).*sub gerente.*': 2,
    '(?i).*gerente.*': 1
}, regex=True)
df_directorio.sort_values(by=['OFICINA','Puesto'], inplace=True, ignore_index=True)
df_directorio.reset_index(drop=True, inplace=True)
df_directorio.head(10)

,OFICINA,Puesto,Correo,Registro,Nombre
0,2,1,TEYZAGUIRRE@BBVA.COM,P015142,"EYZAGUIRRE GONZALES, TATIANA SABRINA"
1,2,2,DMALAGA@BBVA.COM,P018665,"MALAGA MONCADA, DAVID JESUS"
2,5,1,JVEGAC@BBVA.COM,P018307,"VEGA CANCINO, JENNIFER"
3,5,2,KGALVEZ@BBVA.COM,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
4,10,1,EVIVAS@BBVA.COM,P014618,"VIVAS CUZCANO, EVER ROY"
5,10,2,RICHARD.ORDAYA@BBVA.COM,P024186,"ORDAYA QUISPE, RICHARD JHONY"
6,13,1,MGUTIERREZC@BBVA.COM,P008766,"GUTIERREZ CECCARELLI, MANUEL PIEZZINI"
7,13,2,JJOFERNANDEZ@BBVA.COM,P012584,"FERNANDEZ SANDOVAL, JAIR JOAO"
8,14,1,RZEVALLOSV@BBVA.COM,P012428,"ZEVALLOS VASQUEZ, ROBERT ANTONIO"
9,14,2,LAQUINO@BBVA.COM,P013132,"AQUINO LOPEZ, LIZETH NANCY"


In [12]:
def concatenar_correos(grupo: pd.DataFrame) -> pd.Series:
    gerente_correo = grupo.loc[grupo['Puesto']==1, 'Correo'].values
    subgerente_correo = grupo.loc[grupo['Puesto']==2, 'Correo'].values
    
    if len(gerente_correo) > 0 and len(subgerente_correo) > 0:
        correos_concatenados = gerente_correo[0] + ', ' + subgerente_correo[0]
    elif len(gerente_correo) > 0:
        correos_concatenados = gerente_correo[0]
    else:
        correos_concatenados = subgerente_correo[0]
    
    return pd.Series({
        'OFICINA': grupo['OFICINA'].iloc[0],
        'Puesto': 'GERENTE DE OFICINA',
        'Correo': correos_concatenados,
        'Registro': grupo['Registro'].iloc[0],
        'Nombre': grupo['Nombre'].iloc[0]
        })

df_final = df_directorio.groupby('OFICINA').apply(concatenar_correos).reset_index(drop=True)

print(df_final.shape)
df_final.head(3)

(289, 5)


,OFICINA,Puesto,Correo,Registro,Nombre
0,2,GERENTE DE OFICINA,"TEYZAGUIRRE@BBVA.COM, DMALAGA@BBVA.COM",P015142,"EYZAGUIRRE GONZALES, TATIANA SABRINA"
1,5,GERENTE DE OFICINA,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM",P018307,"VEGA CANCINO, JENNIFER"
2,10,GERENTE DE OFICINA,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM",P014618,"VIVAS CUZCANO, EVER ROY"


In [13]:
df_actualizado = pd.merge(df_base, df_final, on='OFICINA', how='left')
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 11)


,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro_x,Nombre_x,Puesto,Correo,Registro_y,Nombre_y
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, eyttus...",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA",GERENTE DE OFICINA,"TEYZAGUIRRE@BBVA.COM, DMALAGA@BBVA.COM",P015142,"EYZAGUIRRE GONZALES, TATIANA SABRINA"
1,5,"GCASTROP@BBVA.COM, buzon-procesal-sin-garantia...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA",GERENTE DE OFICINA,"JVEGAC@BBVA.COM, KGALVEZ@BBVA.COM",P018307,"VEGA CANCINO, JENNIFER"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com, paul...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA",GERENTE DE OFICINA,"EVIVAS@BBVA.COM, RICHARD.ORDAYA@BBVA.COM",P014618,"VIVAS CUZCANO, EVER ROY"


In [14]:
df_actualizado.isna().sum()

OFICINA            0
CORREOS            0
Territorio         0
Nombre Oficina     0
Cargo              0
Registro_x         0
Nombre_x           0
Puesto            52
Correo            52
Registro_y        52
Nombre_y          52
dtype: int64

In [15]:
df_actualizado['Registro'] = df_actualizado['Registro_y'].fillna(df_actualizado['Registro_x'])
df_actualizado['Nombre'] = df_actualizado['Nombre_y'].fillna(df_actualizado['Nombre_x'])
df_actualizado.drop(columns=['Registro_x', 'Registro_y', 'Nombre_x', 'Nombre_y'], inplace=True)

In [16]:
df_actualizado['CORREOS'] = df_actualizado['Correo'].fillna(df_actualizado['CORREOS'])
df_actualizado.drop(columns=['Puesto', 'Correo'], inplace=True)
df_actualizado['CORREOS'] = df_actualizado['CORREOS'].str.lower()
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 7)


,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA SABRINA"
1,5,"jvegac@bbva.com, kgalvez@bbva.com",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P018307,"VEGA CANCINO, JENNIFER"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P014618,"VIVAS CUZCANO, EVER ROY"


In [17]:
df_actualizado['OFICINA'] = df_actualizado['OFICINA'].astype(str).str.zfill(4)
df_actualizado.head(3)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,0002,"teyzaguirre@bbva.com, dmalaga@bbva.com",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA SABRINA"
1,0005,"jvegac@bbva.com, kgalvez@bbva.com",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P018307,"VEGA CANCINO, JENNIFER"
2,0010,"evivas@bbva.com, richard.ordaya@bbva.com",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P014618,"VIVAS CUZCANO, EVER ROY"


In [18]:
df_actualizado.isna().sum()

OFICINA           0
CORREOS           0
Territorio        0
Nombre Oficina    0
Cargo             0
Registro          0
Nombre            0
dtype: int64

In [19]:
correos_nuevos = ['mirko.iriarte@bbva.com', 'paula.zuniga@bbva.com']

def agregar_correos(lista_correos: list) -> str:
    for correo in correos_nuevos:
        if correo not in lista_correos:
            lista_correos.append(correo)
    return ', '.join(lista_correos)

df_actualizado['CORREOS'] = df_actualizado['CORREOS'].str.split(', ').apply(agregar_correos)
df_actualizado.head(3)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,0002,"teyzaguirre@bbva.com, dmalaga@bbva.com, mirko....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA SABRINA"
1,0005,"jvegac@bbva.com, kgalvez@bbva.com, mirko.iriar...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P018307,"VEGA CANCINO, JENNIFER"
2,0010,"evivas@bbva.com, richard.ordaya@bbva.com, mirk...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P014618,"VIVAS CUZCANO, EVER ROY"


In [20]:
df_actualizado.to_excel('CORREOS.xlsx', index=False)
os.startfile('CORREOS.xlsx')